In [ ]:
import stats_functions as sf
import emission.storage.timeseries.aggregate_timeseries as estag
import emission.storage.timeseries.timequery as estt
import arrow
import emission.core.get_database as edb
from emission.core.wrapper.user import User
import matplotlib.pyplot as plt
from datetime import datetime, tzinfo, timedelta
import pandas as pd

import datetime
import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib

class UTC(tzinfo):
    def utcoffset(self, dt):
         return timedelta(0)
    def tzname(self, dt):
        return "UTC"
    def dst(self, dt):
        return timedelta(0)

In [ ]:
agts = estag.AggregateTimeSeries()
sep_dec_tq_data_ts = estt.TimeQuery("data.ts", arrow.get("2018-3-28").timestamp, arrow.now().timestamp)
client_nav_events_df = agts.get_data_df("stats/client_nav_event", time_query=sep_dec_tq_data_ts)
client_nav_events_df.head()

In [ ]:
app_launched_events = client_nav_events_df[client_nav_events_df.name == "opened_app"]

In [ ]:
information_hm = {}
emotion_hm = {}
control_hm = {}
no_clients = set()
for i in range(len(app_launched_events)):
    uuid = app_launched_events[i]['user_id']
    timestamp = app_launched_events[i]['ts']
    pt = datetime.datetime.fromtimestamp(float(timestamp))
    try:
        client = edb.get_profile_db().find_one({"user_id": uuid})['client']
        if client == 'urap-2017-information':
            if uuid in information_hm:
                information_hm[uuid].append(pt)
            else:
                information_hm[uuid] = [pt]
        elif client == 'urap-2017-emotion':
            if uuid in emotion_hm:
                emotion_hm[uuid].append(pt)
            else:
                emotion_hm[uuid] = [pt]
        elif client == 'urap-2017-control':
            if uuid in control_hm:
                control_hm[uuid].append(pt)
            else:
                control_hm[uuid] = [pt]
    except:
        no_clients.add(str(uuid))
for elem in no_clients:
    print(elem + " doesnt have a client for some reason")

In [ ]:
information_data = {}
for u in information_hm:
    earliest = datetime.datetime(2020, 12, 2, 9, 30)
    latest = datetime.datetime(2009, 12, 2, 9, 30)
    for ts in information_hm[u]:
        if ts > latest and ts < datetime.datetime(2018, 6, 6, 0, 0):
            latest = ts
        if ts < earliest:
            earliest = ts
    information_data[u] = (earliest, latest)

emotion_data = {}
for u in emotion_hm:
    earliest = datetime.datetime(2020, 12, 2, 9, 30)
    latest = datetime.datetime(2009, 12, 2, 9, 30)
    for ts in emotion_hm[u]:
        if ts > latest and ts < datetime.datetime(2018, 6, 6, 0, 0):
            latest = ts
        if ts < earliest:
            earliest = ts
    emotion_data[u] = (earliest, latest)

control_data = {}
for u in control_hm:
    earliest = datetime.datetime(2020, 12, 2, 9, 30)
    latest = datetime.datetime(2009, 12, 2, 9, 30)
    for ts in control_hm[u]:
        if ts > latest and ts < datetime.datetime(2018, 6, 6, 0, 0):
            latest = ts
        if ts < earliest:
            earliest = ts
    control_data[u] = (earliest, latest)

In [ ]:
font = {'family' : 'serif'}

matplotlib.rc('font', **font)

def plot_interval_data(data, title):
    event_start = []
    event_finish = []
    users = []

    i = 0
    for u in data:
        users.append(i)
        event_start.append(data[u][0])
        event_finish.append(data[u][1])
        i += 1
    # dates for xaxis
    # translate times and dates lists into matplotlib date format numpy arrays
    start = np.fromiter((mdates.date2num(event) for event in event_start), dtype = 'float', count = len(event_start))
    finish = np.fromiter((mdates.date2num(event) for event in event_finish), dtype = 'float', count = len(event_finish))

    # calculate events durations
    duration = finish - start
    print("Mean: ", np.mean(duration))
    print("Variance: ", np.var(duration))
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    # use errorbar to represent event duration
    ax.errorbar(users, start, [np.zeros(len(duration)), duration], linestyle = '')
    # make matplotlib treat both axis as times
    #ax.xaxis_date()
    ax.yaxis_date()
    plt.title(title)
    plt.xlabel('User')
    plt.ylabel('Date')
    plt.show()

In [ ]:
plot_interval_data(information_data, "Information")

In [ ]:
plot_interval_data(emotion_data, "Emotion")

In [ ]:
plot_interval_data(control_data, "Control")

# Permutation Tests

In [ ]:
def timedelta_to_seconds(td):
    return td.days * 86400 + td.seconds + td.microseconds / 10**6

info_duration = {key: timedelta_to_seconds(val[1]-val[0]) for (key,val) in information_data.items()}
emt_duration = {key: timedelta_to_seconds(val[1]-val[0]) for (key,val) in emotion_data.items()}
ctrl_duration = {key: timedelta_to_seconds(val[1]-val[0]) for (key,val) in control_data.items()}

In [ ]:
# Create a dataframe with columns user_id, , number notifications opened, week number, and group.
df = pd.DataFrame()
for k,v in info_duration.items():
    df = df.append({
                    'uuid': k, 
                    'duration': v, 
                    'group': "information"
                   }, ignore_index=True)
for k,v in emt_duration.items():
    df = df.append({
                    'uuid': k, 
                    'duration': v, 
                    'group': "emotion"
                   }, ignore_index=True)
for k,v in ctrl_duration.items():
    df = df.append({
                    'uuid': k, 
                    'duration': v, 
                    'group': "control"
                   }, ignore_index=True)

In [ ]:
e_c = df[df['group'] != 'information']
sf.print_error_percent(sf.perm_test(e_c['group'], e_c['duration'], sf.mean_diff, 100000), 100000)

In [ ]:
i_c = df[df['group'] != 'emotion']
sf.print_error_percent(sf.perm_test(i_c['group'], i_c['duration'], sf.mean_diff, 100000), 100000)

In [ ]:
i_e = df[df['group'] != 'control']
sf.print_error_percent(sf.perm_test(i_e['group'], i_e['duration'], sf.mean_diff, 100000), 100000)

# Bootstrapping Tests

In [ ]:
e_c = df[df['group'] != 'information']
sf.print_error_percent(sf.bootstrap_test(e_c['group'], e_c['duration'], sf.mean_diff, 100000), 100000)

In [ ]:
i_e = df[df['group'] != 'control']
sf.print_error_percent(sf.bootstrap_test(i_e['group'], i_e['duration'], sf.mean_diff, 100000), 100000)

# Mann Whitney U Tests

In [ ]:
from scipy.stats import mannwhitneyu

control = df[df['group'] == 'control']
control_array = control.as_matrix(columns=control.columns[0:1])

info = df[df['group'] == 'information']
info_array = info.as_matrix(columns=info.columns[0:1])

emotion = df[df['group'] == 'emotion']
emotion_array = emotion.as_matrix(columns=emotion.columns[0:1])

print(mannwhitneyu(emotion_array, control_array))
print(mannwhitneyu(info_array, emotion_array))